In [3]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
from langgraph.checkpoint.memory import InMemorySaver

In [4]:
load_dotenv()
llm =ChatGoogleGenerativeAI(model='gemini-2.5-flash')


In [5]:
class JokeState(TypedDict):

    topic: str
    joke: str
    explanation: str

In [6]:
def generate_joke(state: JokeState):

    prompt = f'generate a joke on the topic {state["topic"]}'
    response = llm.invoke(prompt).content

    return {'joke': response}

def generate_explanation(state: JokeState):

    prompt = f'write an explanation for the joke - {state["joke"]}'
    response = llm.invoke(prompt).content

    return {'explanation': response}

In [ ]:
graph = StateGraph(JokeState)

graph.add_node('generate_joke', generate_joke)
graph.add_node('generate_explanation', generate_explanation)

graph.add_edge(START, 'generate_joke')
graph.add_edge('generate_joke', 'generate_explanation')
graph.add_edge('generate_explanation', END)

checkpointer = InMemorySaver() #this will ensure that all state values will get store at checkpoints

workflow = graph.compile(checkpointer=checkpointer)

In [ ]:
config1 = {"configurable": {"thread_id": "1"}}
config2 = {"configurable": {"thread_id": "2"}}
workflow.invoke({'topic':'pizza'}, config=config2) #state value will get saved in corresponding to thread id2

{'topic': 'pizza',
 'joke': 'Why did the pizza chef go broke?\nBecause he kept *making* too much dough!',
 'explanation': 'This joke is a classic pun that plays on the double meaning of the word "dough."\n\nHere\'s the breakdown:\n\n1.  **"Dough" (Meaning 1 - Literal):** In the context of a pizza chef, "dough" refers to the mixture of flour, water, and yeast that is flattened and topped to make a pizza. A chef *makes* dough as part of their job.\n\n2.  **"Dough" (Meaning 2 - Slang):** "Dough" is also a common slang term for **money**. If someone has a lot of "dough," they\'re rich. If you need "dough," you need money.\n\n**How the joke works:**\n\n*   When you hear, "Why did the pizza chef go broke?", your mind immediately thinks about financial problems, and therefore, "dough" in the sense of **money**.\n*   The punchline, "Because he kept *making* too much dough!", deliberately plays on this expectation. Instead of implying he literally made too much *money* (which would make him ric

In [ ]:
workflow.get_state(config1) # it prints final state VALUE for config1 which has thread id 1 hence pasta joke

StateSnapshot(values={'topic': 'pasta', 'joke': 'Why did the pasta break up with the sauce?\n\nBecause it was an **impasta**!', 'explanation': 'This is a classic example of a **pun**, which relies on wordplay for its humor.\n\nHere\'s the breakdown:\n\n1.  **The Setup:** "Why did the pasta break up with the sauce?" This gives human characteristics (having a relationship, breaking up) to food items. This is a common setup for jokes, especially puns.\n\n2.  **The Punchline:** "Because it was an **impasta**!"\n    *   **"Impasta"** sounds exactly like the word **"impostor."**\n    *   An **impostor** is someone who pretends to be someone else to deceive them; a fraud or a fake.\n\n3.  **The Joke\'s Logic:** The humor comes from combining these ideas. The pasta was acting like an "impostor" – meaning it was being fake, not genuine, or perhaps pretending to be something it wasn\'t in its "relationship" with the sauce. And just like a person might break up with someone who isn\'t genuine, th

In [23]:
list(workflow.get_state_history(config2)) #it shows both final intermediate state values for config 2 means thread_id 2 hence pizza joke

[StateSnapshot(values={'topic': 'pizza', 'joke': 'Why did the pizza chef go broke?\nBecause he kept *making* too much dough!', 'explanation': 'This joke is a classic pun that plays on the double meaning of the word "dough."\n\nHere\'s the breakdown:\n\n1.  **"Dough" (Meaning 1 - Literal):** In the context of a pizza chef, "dough" refers to the mixture of flour, water, and yeast that is flattened and topped to make a pizza. A chef *makes* dough as part of their job.\n\n2.  **"Dough" (Meaning 2 - Slang):** "Dough" is also a common slang term for **money**. If someone has a lot of "dough," they\'re rich. If you need "dough," you need money.\n\n**How the joke works:**\n\n*   When you hear, "Why did the pizza chef go broke?", your mind immediately thinks about financial problems, and therefore, "dough" in the sense of **money**.\n*   The punchline, "Because he kept *making* too much dough!", deliberately plays on this expectation. Instead of implying he literally made too much *money* (whic

Time Travel

In [ ]:
workflow.get_state({"configurable": {"thread_id": "2", "checkpoint_id": "1f0dd7b0-1256-6a4f-8000-acadefcd1518"}})

StateSnapshot(values={'topic': 'pasta'}, next=('generate_joke',), config={'configurable': {'thread_id': '2', 'checkpoint_id': '1f0dd7b0-1256-6a4f-8000-acadefcd1518'}}, metadata={'source': 'loop', 'step': 0, 'parents': {}}, created_at='2025-12-20T08:08:01.843039+00:00', parent_config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1f0dd7b0-1253-6f7d-bfff-af70370e2f59'}}, tasks=(PregelTask(id='cce00e6a-0079-b7ba-4565-dbab7fb0120c', name='generate_joke', path=('__pregel_pull', 'generate_joke'), error=None, interrupts=(), state=None, result={'joke': 'Why did the pasta get arrested?\n\nBecause it was an **impasta**!'}),), interrupts=())

In [26]:
workflow.invoke(None, {"configurable":{"thread_id": "2", "checkpoint_id": "1f0dd7b0-1256-6a4f-8000-acadefcd1518"}})

{'topic': 'pasta',
 'joke': 'Why did the pasta get in trouble?\n\nBecause it was an **impasta**!',
 'explanation': 'This is a classic pun-based joke!\n\nIt works by playing on the similarity in sound between two words: **"impasta"** and **"impostor"**.\n\nLet\'s break it down:\n\n1.  **Impostor:** This is the real word. An impostor is a person who pretends to be someone else in order to deceive others, especially for fraudulent purposes. Someone who is an impostor would definitely "get in trouble."\n\n2.  **Impasta:** This is the made-up word for the joke. It takes the word "pasta" (the food) and adds the prefix "im-" to make it sound exactly like "impostor."\n\n**The Joke\'s Humor:**\nThe humor comes from the absurd substitution. We\'re talking about actual pasta (the food), but the punchline gives it the characteristics of an "impostor" because of the sound-alike word. It\'s unexpected and silly to imagine a bowl of spaghetti pretending to be someone else and getting arrested for it!

updating state

In [27]:
workflow.update_state({"configurable": {"thread_id": "1", "checkpoint_id": "1f0dd7b0-1256-6a4f-8000-acadefcd1518", "checkpoint_ns": ""}}, {'topic':'samosa'})

{'configurable': {'thread_id': '1',
  'checkpoint_ns': '',
  'checkpoint_id': '1f0ddb67-6f9c-63e8-8000-574f83bbb183'}}